In [164]:
# Here are the libraries I am using
import pandas as pd 
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from os import path
import json
import csv
import twitter
from datetime import datetime
from datetime import date

In [165]:
from __future__ import print_function, division
import time

In [166]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

from sklearn.model_selection import GridSearchCV 

In [167]:
t0 = time.time()
df_o= pd.read_csv('out2017_1.csv')
print("%.2f secs ==> pd.read_csv"%(time.time()-t0))

0.03 secs ==> pd.read_csv


In [168]:
#remove all the rows that contain a missing value
df = df_o.dropna()

In [169]:
df.head(5)

,screen_name,device_count,fav_avg,fav_max,fav_std,fav_total,len_mean,rt_avg,rt_max,rt_std,rt_total,time_delta,follower,friends,eleted
0,Termy,5,0.381295,8.0,0.916638,53,116.748201,349.230216,11542.0,1582.071506,48543,0.231884,415.0,540.0,0
1,JamesWrightLD,5,1.746988,48.0,5.461461,145,128.084337,131.024096,3315.0,414.142400,10875,0.390244,738.0,1427.0,1
2,WilliamPHDyer,4,1.263158,22.0,3.804551,72,111.333333,68.280702,1682.0,244.181968,3892,0.535714,1673.0,891.0,0
3,DanielZeichner,6,12.599057,225.0,28.140393,2671,128.735849,35.500000,2293.0,181.151440,7526,0.151659,5000.0,4281.0,1
4,robertcourts,4,9.512195,28.0,7.587287,390,133.609756,109.463415,1712.0,362.053432,4488,0.775000,5000.0,3190.0,0


In [170]:
#Stratified sampling method by the value of Stance
def split_to_train_test(df, label_column, frac=0.7):
    train_df, val_df = pd.DataFrame(), pd.DataFrame()
    labels = df[label_column].unique()
    for lbl in labels:
        lbl_df = df[df[label_column] == lbl]
        lbl_train_df = lbl_df.sample(frac=frac)
        lbl_val_df = lbl_df.drop(lbl_train_df.index)
        print ('\n%s:\n----\ntotal:%d\ntrain_df:%d\nval_df:%d' % (lbl, len(lbl_df), len(lbl_train_df), len(lbl_val_df)))
        train_df = train_df.append(lbl_train_df)
        val_df = val_df.append(lbl_val_df)

    return train_df, val_df



In [171]:
print('After staratified sampling, get two sets: training subset and validation subset')
train, val = split_to_train_test(df, 'eleted', 0.9)

After staratified sampling, get two sets: training subset and validation subset

0:
----
total:1144
train_df:1030
val_df:114

1:
----
total:405
train_df:364
val_df:41


In [172]:
train.head(5)

,screen_name,device_count,fav_avg,fav_max,fav_std,fav_total,len_mean,rt_avg,rt_max,rt_std,rt_total,time_delta,follower,friends,eleted
79,GonulDaniels,3,4.764706,50.0,8.114833,810,119.970588,53.652941,2128.0,207.293462,9121,0.189349,2028.0,2564.0,0
1562,tonygouldukip,1,0.004219,1.0,0.064820,1,123.531646,272.476793,8821.0,973.377861,64577,0.033898,3.0,13.0,0
1351,seanddundeewest,2,0.000000,0.0,0.000000,0,101.769231,0.076923,1.0,0.266469,1,0.083333,9.0,0.0,0
1046,NazShahBfd,3,9.328358,215.0,21.763236,2500,117.451493,680.608209,18095.0,2270.853352,182403,0.119850,5000.0,935.0,0
1090,LSRPlaid,3,6.213675,70.0,12.558805,727,126.512821,144.153846,14683.0,1350.349101,16866,0.060345,5000.0,1841.0,0


In [173]:
val.head(5)

,screen_name,device_count,fav_avg,fav_max,fav_std,fav_total,len_mean,rt_avg,rt_max,rt_std,rt_total,time_delta,follower,friends,eleted
4,robertcourts,4,9.512195,28.0,7.587287,390,133.609756,109.463415,1712.0,362.053432,4488,0.775000,5000.0,3190.0,0
20,racheleden,4,1.890052,46.0,4.389833,361,112.198953,485.204188,27146.0,2895.491239,92674,0.168421,3077.0,2526.0,0
76,InclusionQuery,3,0.000000,0.0,0.000000,0,58.750000,0.000000,0.0,0.000000,0,1.000000,243.0,172.0,0
97,crimicans98,1,8.250000,129.0,23.615408,231,107.071429,43.964286,617.0,132.060722,1231,1.185185,942.0,553.0,0
114,kev927riddy,2,0.185185,4.0,0.630718,15,109.432099,88.839506,4898.0,565.886685,7196,0.400000,1172.0,2195.0,0


In [174]:
parameters = {  
    'penalty':('l1', 'l2'),   
    'C':[1,2,3,4],   
} 

def preprocess(df, features):
    X = df[features].values
    
    X = scaler.fit_transform(X)
    return X

scaler = StandardScaler()

clf = LogisticRegression(penalty='l2')


features = ['device_count', 'fav_avg', 'fav_max', 'fav_std',
            'fav_total', 'len_mean',  'rt_avg','rt_std','rt_total','time_delta','follower','friends']
#bidprice,payprice去掉

y_train = train['eleted'].values
y_val = val['eleted'].values

X_train = preprocess(train, features)
X_val = preprocess(val, features)
 
y_val_predict = clf.fit(X_train, y_train).predict(X_val)
print(classification_report(y_val, y_val_predict))

             precision    recall  f1-score   support

          0       0.77      0.96      0.85       114
          1       0.64      0.22      0.33        41

avg / total       0.74      0.76      0.72       155



In [175]:
from sklearn.model_selection import cross_val_score

In [176]:
from sklearn import svm
from sklearn import metrics

In [154]:
from sklearn.model_selection import cross_val_predict

clf=clf = LogisticRegression(penalty='l2')

predicted = cross_val_predict(clf, X_val, y_val, cv=10)
metrics.accuracy_score(y_val, y_val_predict) 
#one year

0.78260869565217395

In [137]:
from sklearn.model_selection import cross_val_predict

clf=clf = LogisticRegression(penalty='l2')

predicted = cross_val_predict(clf, X_val, y_val, cv=10)
metrics.accuracy_score(y_val, y_val_predict) 
#two years

0.80952380952380953

In [123]:
from sklearn.model_selection import cross_val_predict

clf=clf = LogisticRegression(penalty='l2')

predicted = cross_val_predict(clf, X_val, y_val, cv=10)
metrics.accuracy_score(y_val, y_val_predict) 
#3 months

0.73584905660377353

In [177]:
from sklearn.model_selection import cross_val_predict

clf=clf = LogisticRegression(penalty='l2')

predicted = cross_val_predict(clf, X_val, y_val, cv=10)
metrics.accuracy_score(y_val, y_val_predict) 
#1 months

0.76129032258064511